<a href="https://colab.research.google.com/github/Sparrow0hawk/RAMP-UA/blob/develop-AC-updateNotebook/examples/colab/ramp-colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sensitivity Analysis with the OpenCL RAMP model

This is an example notebook that runs on Google Colab that makes use of the RAMP-UA OpenCL model.


In [1]:
! curl -O https://raw.githubusercontent.com/Urban-Analytics/RAMP-UA/develop-AC-updateNotebook/examples/colab/ramp-colab.sh
!bash ramp-colab.sh data-migration

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2417  100  2417    0     0  16554      0 --:--:-- --:--:-- --:--:-- 16554
Setting up your colab environment to run RAMP-UA
********************************************************************************************************
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  git-lfs
0 upgraded, 1 newly installed, 0 to remove and 13 not upgraded.
Need to get 2,129 kB of archives.
After this operation, 7,662 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 git-lfs amd64 2.3.4-1 [2,129 kB]
Fetched 2,129 kB in 1s (2,917 kB/s)
Selecting previously unselected package git-lfs.
(Reading database ... 146374 files and directories currently installed.)
Preparing to unpack .../git-lfs_2.3.4

In [2]:
import sys
sys.path.append('/usr/local/lib/python3.6/site-packages')

### Import opencl modules

In [3]:
# Useful for connecting to this kernel
import os
import numpy as np
import matplotlib.pyplot as plt
from microsim.opencl.ramp.run import run_headless
from microsim.opencl.ramp.snapshot_convertor import SnapshotConvertor
from microsim.opencl.ramp.snapshot import Snapshot
from microsim.opencl.ramp.params import Params, IndividualHazardMultipliers, LocationHazardMultipliers
from microsim.opencl.ramp.simulator import Simulator
from microsim.opencl.ramp.disease_statuses import DiseaseStatus

from opencl_runner import OpenCLRunner # Some additional notebook-specific functions required (functions.py)

%connect_info

{
  "shell_port": 35821,
  "iopub_port": 59393,
  "stdin_port": 42673,
  "control_port": 46483,
  "hb_port": 58003,
  "ip": "127.0.0.1",
  "key": "",
  "transport": "tcp",
  "signature_scheme": "hmac-sha256",
  "kernel_name": ""
}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing kernel-783e7122-e070-4d77-898c-e5e7fc9301bc.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.


### Setup params for all runs

Read the parameters file

In [4]:
PARAMETERS_FILE = os.path.join("RAMP-UA","model_parameters", "default.yml")
PARAMS = OpenCLRunner.create_parameters(parameters_file=PARAMETERS_FILE)

In [5]:
%%bash

cd RAMP-UA

python microsim/main.py -init -ocl

Reading parameters file: ./model_parameters/default.yml. Any other model-related command-line arguments are being ignored
Running model with the following parameters:
	Parameters file: ./model_parameters/default.yml
	Scenario directory: BaseScenario
	Initialise (and then exit?): True
	Number of iterations: 80
	Data dir: devon_data
	Using quant data? yes: QUANT_RAMP
	Outputting results?: True
	Outputting results at every iteration?: False
	Debug mode?: False
	Number of repetitions: 1
	Lockdown file: google_mobility_lockdown_daily_14_day_moving_average.csv
 	Use cache?: True
 	Use OpenCL version?: True
 	Use OpenCL GUI?: False
 	Use OpenCL GPU for processing?: False
 	Calibration parameters: {'hazard_individual_multipliers': {'presymptomatic': 1, 'asymptomatic': 0.75, 'symptomatic': 1.0}, 'hazard_location_multipliers': {'Retail': 1.0, 'PrimarySchool': 1.0, 'SecondarySchool': 1.0, 'Home': 1.0, 'Work': 1.0}, 'risk_multiplier': 1.0}
 	Disease parameters: {'current_risk_beta': 0.003, 'risk_c

Warning message:
In options(stringsAsFactors = TRUE) :
  'options(stringsAsFactors = TRUE)' is deprecated and will be disabled
tcmalloc: large alloc 3107332096 bytes == 0x55970e256000 @  0x7f5cdb7a51e7 0x55964ccdb08b 0x55964cd99a1f 0x55964cde718e 0x55964ce0a3ed 0x55964ce38caa 0x55964cdfb4b5 0x55964ce0a3af 0x55964ce38caa 0x55964cd8074b 0x55964cdfb933 0x55964ce0a3af 0x55964ce38caa 0x55964cd8074b 0x55964cdfb933 0x55964ce0a3af 0x55964ce38caa 0x55964cd8074b 0x55964cdfb933 0x55964ce0a3af 0x55964ce39ace 0x55964cdc02b4 0x55964ce0a3af 0x55964ce38caa 0x55964cd8074b 0x55964ce0b101 0x55964cd865ae 0x55964ce3a3a2 0x55964cd8074b 0x55964cd8209a 0x55964ce0ae41
tcmalloc: large alloc 1264861184 bytes == 0x55970e6fc000 @  0x7f5cdb7a51e7 0x55964ccdb08b 0x55964cd99a1f 0x55964ce137b3 0x55964cdd2b77 0x55964cd824f1 0x7f5cda0e97d4 0x7f5cda0ea0c6 0x7f5cda0ee336 0x7f5cda0f6475 0x55964ce0a585 0x55964ce38caa 0x55964cdfb4b5 0x55964ce0a3af 0x55964ce38caa 0x55964cd8074b 0x55964cd81f4d 0x55964ce0ae41 0x55964ce30007 0x5

In [6]:
OPENCL_DIR = os.path.join("RAMP-UA","microsim","opencl")
SNAPSHOT_FILEPATH = os.path.join(OPENCL_DIR,"snapshots","cache.npz")
assert os.path.isfile(SNAPSHOT_FILEPATH), f"Snapshot doesn't exist: {SNAPSHOT_FILEPATH}"

### Get snapshot path
**NB** this is the path to the OpenCL snapshot file generated by running `microsim/main.py`. To run with new population data just re-run `main.py --opencl` without the `--use-cache` option, so that it regenerates a new snapshot file and writes it to this location.

In [7]:
import pyopencl 

platforms = pyopencl.get_platforms()

for i, platform in enumerate(platforms):
    devices = platform.get_devices()
    
    print(devices)


[<pyopencl.Device 'Tesla T4' on 'NVIDIA CUDA' at 0x31e1bd0>]


## Run OpenCL simulation for multiple repetitions

In [8]:
results = OpenCLRunner.run_opencl_model_multi(
    repetitions=5, 
    iterations=100, 
    params=OpenCLRunner.create_parameters(parameters_file=PARAMETERS_FILE),
    use_gpu=True,
    store_detailed_counts=True,
    opencl_dir=OPENCL_DIR,
    snapshot_filepath=SNAPSHOT_FILEPATH,
    multiprocess=False
)

summaries = [x[0] for x in results]
final_results = [x[1] for x in results]
print("Finished")

Running models: 100%|██████████| 5/5 [00:46<00:00,  9.26s/it]

.. finished, took 46.31s)
Finished


## Plot output summary data

### Total counts of disease status

In [9]:
def plot_summaries(summaries, plot_type="error_bars"):

    #fig, ax = plt.subplots(1, len(DiseaseStatus), sharey=True)
    fig, ax = plt.subplots(1, 1, figsize=(10,7))
    
    # Work out the number of repetitions and iterations
    iters, reps = _get_iters_and_reps(summaries)
    x = range(iters)

    for d, disease_status in enumerate(DiseaseStatus):
        if disease_status==DiseaseStatus.Susceptible or disease_status==DiseaseStatus.Recovered:
            continue
        # Calculate the mean and standard deviation
        matrix = np.zeros(shape=(reps,iters))
        for rep in range(reps):
            matrix[rep] = summaries[rep].total_counts[d]
        mean = np.mean(matrix, axis=0)
        sd = np.std(matrix, axis=0)
        if plot_type == "error_bars":
            ax.errorbar(x, mean, sd, label=f"{disease_status}" )
        elif plot_type == "lines":
            for rep in range(reps):
                ax.plot(x, matrix[rep], label=f"{disease_status} {rep}", 
                        color=plt.cm.get_cmap("hsv", len(DiseaseStatus))(d) )
                
    ax.legend() 
    ax.set_title("Disease Status")
    ax.set_xlabel("Iteration")
    ax.set_ylabel("Number of cases")

def _get_iters_and_reps(summaries):
    reps = len(summaries)
    iters = len(summaries[0].total_counts[0])
    return (iters, reps)

In [ ]:
plot_summaries(summaries=summaries, plot_type="error_bars")

### Disease statuses by age

In [ ]:
   
def plot_disease_status_by_age(summaries):

    #fig, ax = plt.subplots(1, len(DiseaseStatus), sharey=True)
    fig, ax = plt.subplots(int(len(DiseaseStatus)/2), int(len(DiseaseStatus)/2), 
                           figsize=(15,11), tight_layout=True)
    iters, reps = _get_iters_and_reps(summaries)
    x = range(iters)
    age_thresholds = summaries[0].age_thresholds

    for d, disease_status in enumerate(DiseaseStatus):
        lower_age_bound = 0
        for age_idx in range(len(age_thresholds)):
            matrix = np.zeros(shape=(reps, iters))
            for rep in range(reps):
                #matrix[age_idx][rep][it] = summaries[rep].age_counts[str(disease_status)][age_idx][it]
                matrix[rep] = summaries[rep].age_counts[str(disease_status)][age_idx]
            mean = np.mean(matrix, axis=0)
            sd = np.std(matrix, axis=0)
            ax.flat[d].errorbar(x, mean, sd, label=f"{lower_age_bound} - {age_thresholds[age_idx]}" )
            lower_age_bound = age_thresholds[age_idx]
                
            ax.flat[d].legend() 
            ax.flat[d].set_title(f"{str(disease_status)}")
            ax.flat[d].set_xlabel("Iteration")
            ax.flat[d].set_ylabel("Number of cases")
    #fig.set_title(f"Num {disease_status} people by age group")

In [ ]:
plot_disease_status_by_age(summaries)

### Plot MSOA geodata

#### Load MSOA shapes

In [ ]:
from microsim.load_msoa_locations import load_osm_shapefile, load_msoa_shapes
import pandas as pd

data_dir = ("RAMP-UA/devon_data")

osm_buildings = load_osm_shapefile(data_dir)

devon_msoa_shapes = load_msoa_shapes(data_dir, visualize=False)

devon_msoa_shapes.plot()
plt.show()

In [ ]:
import pandas as pd

def plot_msoa_choropleth(msoa_shapes, summary, disease_status, timestep):
    # get dataframes for all statuses
    msoa_data = summary.get_area_dataframes()
    
    msoa_data_for_status = msoa_data[disease_status]

    # add "Code" column so dataframes can be merged
    msoa_data_for_status["Code"] = msoa_data_for_status.index
    msoa_shapes = pd.merge(msoa_shapes, msoa_data_for_status, on="Code")

    msoa_shapes.plot(column=f"Day{timestep}", legend=True)
    plt.show()

### Plot disease status by MSOA for a given timestep and status

In [ ]:
disease_status = "exposed"

plot_msoa_choropleth(devon_msoa_shapes, summaries[0], disease_status, 99)